# Importing python libraries

In [5]:
import numpy as np
import pandas as pd
import re
import html.parser
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Reading the input dataset

In [6]:
dataset = pd.read_csv('C:\\Users\\usury\\Downloads\\Dataset_HOTEL.csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68275 entries, 0 to 68274
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   reviewer_name  68275 non-null  object
 1   reviewer_id    68275 non-null  int64 
 2   listing_id     68275 non-null  int64 
 3   id             68275 non-null  int64 
 4   comments       68222 non-null  object
dtypes: int64(3), object(2)
memory usage: 2.6+ MB


# Fetching the hotel names

In [7]:
datasethotellname = pd.read_csv('C:\\Users\\usury\\Downloads\\hotelnames.csv')

# Assigning hotel names to the dataset

In [11]:
dic={}
index=0
for i in set(dataset["listing_id"]):
    if index< len(set(datasethotellname["name"])):
        if i not in dic:
            dic[int(i)] = list(set(datasethotellname["name"]))[index]
            index+=1
        else:
            pass

# Assigning each listing is with a unique hotel name

In [12]:
for index, rows in dataset.iterrows():

    if rows["listing_id"] in dic:
        dataset.at[index, 'Hotel_name'] = dic[rows["listing_id"]]



    else:
        dataset.at[index, 'Hotel_name'] = 'UNlisted'

# Current dataset with hotel names

In [13]:
dataset

,reviewer_name,reviewer_id,listing_id,id,comments,Hotel_name
0,Olivier,4298113,1178162,4724140,My stay at islam's place was really cool! Good...,UNlisted
1,Charlotte,6452964,1178162,4869189,Great location for both airport and city - gre...,UNlisted
2,Sebastian,6449554,1178162,5003196,We really enjoyed our stay at Islams house. Fr...,UNlisted
3,Marine,2215611,1178162,5150351,The room was nice and clean and so were the co...,UNlisted
4,Andrew,6848427,1178162,5171140,Great location. Just 5 mins walk from the Airp...,UNlisted
...,...,...,...,...,...,...
68270,Antonio,22034145,7462268,80537457,Joe y su mujer son encantadores. La habitación...,UNlisted
68271,Steve,40052513,7462268,83640094,Joe was on his way to Jamaica to be married! o...,UNlisted
68272,Nick,77129134,7462268,85797088,The room was very clean as were the bathrooms ...,UNlisted
68273,Vid,15799803,7462268,97264637,Staying in Lower Allston at Joe and Nancy's pl...,UNlisted


# Dropping unlisted hotel rows from the dataset 

In [8]:
dataset = dataset[dataset.Hotel_name != 'UNlisted']

# Hotel names other than Unlisted

In [14]:
set(dataset['Hotel_name'])

{'Carter Hotel',
 'Red Roof Inn',
 'SUN OAK MUSEUM AND GUEST HOUSE',
 'QUEEN & CRESCENT HOTEL',
 'Days Inn Berea Ky',
 'THE OLD NO. 77 HOTEL & CHANDLERY - MAIN',
 'Hawthorn Suites Dayton North',
 'Concord Apartments By Boq Lodging',
 'Candlewood Suites Lexington',
 'Rodeway Inn',
 'Old Saco Inn Bed & Breakfast',
 'THE ROOSEVELT HOTEL WAC',
 'The Inn At Bella Vista',
 'HUBBARD MANSION BED & BREAKFAST',
 "Fort's Cedar View",
 'Tidewater Inn',
 'Frontier Cabins Motel',
 'Sheraton Baltimore North Hotels',
 'Terrace Inns Suites',
 'Hyatt Place Atlanta/alpharetta/north Point Mall',
 'BUDGET INN EXPRESS',
 'Elba Sara Beach & Golf Resort',
 'Avalon Hotel Group',
 'THE RITZ CARLTON HOTEL CO LLC',
 'Hampton Inn & Suites Crescent City',
 'RITZ-CARLTON GEORGETOWN',
 'WASHINGTON HILTON',
 'SAINT GREGORY',
 'ASTOR CROWN PLAZA NEW ORLEANS ATTN: DIR. OF FIN.',
 'MANDARIN ORIENTAL',
 'BON MAISON GUEST HOUSE',
 'Port-o-call Motel',
 'HAMPTON INN WASHINGTON DC/WHITE HOUSE',
 'Candlewood Suites Oklahoma C

# Displaying the column information/ metadata

In [15]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68275 entries, 0 to 68274
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   reviewer_name  68275 non-null  object
 1   reviewer_id    68275 non-null  int64 
 2   listing_id     68275 non-null  int64 
 3   id             68275 non-null  int64 
 4   comments       68222 non-null  object
 5   Hotel_name     68275 non-null  object
dtypes: int64(3), object(3)
memory usage: 3.1+ MB


# Dropping null rows from the dataset

In [16]:
dataset = dataset.dropna()

# Checking the input comment data

In [20]:
inpdata = dataset.iloc[:,-2].values
inpdata

array(["My stay at islam's place was really cool! Good location, 5min away from subway, then 10min from downtown. The room was nice, all place was clean. Islam managed pretty well our arrival, even if it was last minute ;) i do recommand this place to any airbnb user :)",
       'Great location for both airport and city - great amenities in the house: Plus Islam was always very helpful even though he was away',
       "We really enjoyed our stay at Islams house. From the outside the house didn't look so inviting but the inside was very nice! Even though Islam himself was not there everything was prepared for our arrival. The airport T Station is only a 5-10 min walk away. The only little issue was that all the people in the house had to share one bathroom. But it was not really a problem and it worked out fine. We would recommend Islams place for a stay in Boston. ",
       ...,
       'The room was very clean as were the bathrooms and kitchen. The bed was also very comfortable. Bathro

# Cleansing the dataset

In [21]:
import re
import html.parser
def cleaning_Data(data):
    data = re.sub(r'[?|$|.|!|\n|;|-|,|:|\d+]',r'',data)
    data = html.parser.unescape(data)
    return data

#  Considering only english words

In [22]:
def cleaner(string):
    new_string=re.sub('[^a-zA-Z0-9]',' ',string)
    cleaned_string=re.sub('\s+',' ',new_string)
    return cleaned_string

# Calling the cleansing function to clean the dataset

In [23]:
len1 = inpdata.size
print("Review count",len1)
for i in range(0,len1):
    inpdata[i] = cleaning_Data(inpdata.item(i))
for i in range(0,len1):
    inpdata[i] = cleaner(inpdata.item(i))

Review count 68222


# TFIDF Vectorization 

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorized = TfidfVectorizer(min_df=50,max_features=100,stop_words='english',ngram_range=(1,1))
output_data = vectorized.fit_transform(inpdata.flatten())

# Output data array

In [25]:
from pprint import pprint
pprint(output_data.toarray())

array([[0.        , 0.        , 0.29407059, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.27697292, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])


# Creating K-means clusters from a range of 1-5: 

In [26]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters= 5, init='k-means++', random_state=0)
Y_output=model.fit_predict(output_data)

In [27]:
model.cluster_centers_.shape

(5, 100)

In [28]:
making_clusters =pd.DataFrame(output_data.toarray(), columns=vectorized.get_feature_names())
print(making_clusters.head())


   access  accommodating    airbnb   airport  amazing  apartment      area  \
0     0.0            0.0  0.294071  0.000000      0.0        0.0  0.000000   
1     0.0            0.0  0.000000  0.461113      0.0        0.0  0.000000   
2     0.0            0.0  0.000000  0.243232      0.0        0.0  0.000000   
3     0.0            0.0  0.000000  0.388276      0.0        0.0  0.311325   
4     0.0            0.0  0.000000  0.401491      0.0        0.0  0.000000   

    arrival  arrived      away  ...  sure  thank  thanks  time  trip  visit  \
0  0.309562      0.0  0.298849  ...   0.0    0.0     0.0   0.0   0.0    0.0   
1  0.000000      0.0  0.415135  ...   0.0    0.0     0.0   0.0   0.0    0.0   
2  0.226829      0.0  0.218979  ...   0.0    0.0     0.0   0.0   0.0    0.0   
3  0.000000      0.0  0.000000  ...   0.0    0.0     0.0   0.0   0.0    0.0   
4  0.000000      0.0  0.000000  ...   0.0    0.0     0.0   0.0   0.0    0.0   

       walk  walking  welcoming  wonderful  
0  0.000000

In [29]:
# Add column corresponding to cluster number
making_clusters['Cluster'] = Y_output  
word_frequencies_by_cluster = making_clusters.groupby('Cluster').sum()
word_frequencies_by_cluster=word_frequencies_by_cluster.sort_values(by=['amazing'],ascending=False)

# Map rating to cluster label


In [30]:
making_clusters['Cluster'] = making_clusters['Cluster'].map({4: 1, 3: 2, 2: 3,1:4,0:5})
print(making_clusters.head())

   access  accommodating    airbnb   airport  amazing  apartment      area  \
0     0.0            0.0  0.294071  0.000000      0.0        0.0  0.000000   
1     0.0            0.0  0.000000  0.461113      0.0        0.0  0.000000   
2     0.0            0.0  0.000000  0.243232      0.0        0.0  0.000000   
3     0.0            0.0  0.000000  0.388276      0.0        0.0  0.311325   
4     0.0            0.0  0.000000  0.401491      0.0        0.0  0.000000   

    arrival  arrived      away  ...  thank  thanks  time  trip  visit  \
0  0.309562      0.0  0.298849  ...    0.0     0.0   0.0   0.0    0.0   
1  0.000000      0.0  0.415135  ...    0.0     0.0   0.0   0.0    0.0   
2  0.226829      0.0  0.218979  ...    0.0     0.0   0.0   0.0    0.0   
3  0.000000      0.0  0.000000  ...    0.0     0.0   0.0   0.0    0.0   
4  0.000000      0.0  0.000000  ...    0.0     0.0   0.0   0.0    0.0   

       walk  walking  welcoming  wonderful  Cluster  
0  0.000000      0.0        0.0       

In [31]:
word_frequencies_by_cluster = making_clusters.groupby('Cluster').sum()
word_frequencies_by_cluster = word_frequencies_by_cluster.sort_values(by=['amazing'],ascending=False)
print(word_frequencies_by_cluster)

             access  accommodating      airbnb     airport     amazing  \
Cluster                                                                  
2        469.392174     592.022783  800.484817  573.149680  477.474168   
4        183.925275     301.919767  255.481672  258.167549  331.913059   
1        285.537789     328.879417  275.624935  166.234563  306.895075   
5        153.684081     272.557313  172.355823   97.264743  176.666354   
3          0.639886       0.000000   25.774507    2.696110    1.404194   

           apartment        area      arrival     arrived        away  ...  \
Cluster                                                                ...   
2         243.507661  830.619476   337.400463  531.545409  591.333895  ...   
4         263.110491  373.076988   122.954799  185.759023  257.666335  ...   
1        4533.357206  525.287029   228.611430  294.929821  339.288844  ...   
5         351.017657  208.568501  1095.430898   82.351316   96.522625  ...   
3           3

In [32]:
dataset['rating']=making_clusters.iloc[:,-1:].values
print(dataset.head())

  reviewer_name  reviewer_id  listing_id       id  \
0       Olivier      4298113     1178162  4724140   
1     Charlotte      6452964     1178162  4869189   
2     Sebastian      6449554     1178162  5003196   
3        Marine      2215611     1178162  5150351   
4        Andrew      6848427     1178162  5171140   

                                            comments Hotel_name  rating  
0  My stay at islam s place was really cool Good ...   UNlisted       4  
1  Great location for both airport and city great...   UNlisted       5  
2  We really enjoyed our stay at Islams house Fro...   UNlisted       2  
3  The room was nice and clean and so were the co...   UNlisted       2  
4  Great location Just mins walk from the Airport...   UNlisted       2  


C:\Users\usury\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Dataset with the predicted ratings:

In [33]:
dataset

,reviewer_name,reviewer_id,listing_id,id,comments,Hotel_name,rating
0,Olivier,4298113,1178162,4724140,My stay at islam s place was really cool Good ...,UNlisted,4
1,Charlotte,6452964,1178162,4869189,Great location for both airport and city great...,UNlisted,5
2,Sebastian,6449554,1178162,5003196,We really enjoyed our stay at Islams house Fro...,UNlisted,2
3,Marine,2215611,1178162,5150351,The room was nice and clean and so were the co...,UNlisted,2
4,Andrew,6848427,1178162,5171140,Great location Just mins walk from the Airport...,UNlisted,2
...,...,...,...,...,...,...,...
68270,Antonio,22034145,7462268,80537457,Joe y su mujer son encantadores La habitaci n ...,UNlisted,3
68271,Steve,40052513,7462268,83640094,Joe was on his way to Jamaica to be married on...,UNlisted,2
68272,Nick,77129134,7462268,85797088,The room was very clean as were the bathrooms ...,UNlisted,2
68273,Vid,15799803,7462268,97264637,Staying in Lower Allston at Joe and Nancy s pl...,UNlisted,4


# populating the csv file with the newly generated ratings

In [57]:
dataset.to_csv('Rating_prediction_using_clustering3.csv',columns=['listing_id','Hotel_name','reviewer_id','reviewer_name','rating','comments'],index=False)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31701 entries, 252 to 68242
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   reviewer_name  31701 non-null  object
 1   reviewer_id    31701 non-null  int64 
 2   listing_id     31701 non-null  int64 
 3   id             31701 non-null  int64 
 4   comments       31701 non-null  object
 5   Hotel_name     31701 non-null  object
 6   rating         31701 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 1.9+ MB
